# Scripts Mosaicos

In [1]:
import ee
import geemap
import os
import folium


In [2]:
Map = geemap.Map()
Map2 = geemap.Map()
Map3 = geemap.Map()

## Área de Mosaicos

### Patagonia

In [3]:
work_dir = os.path.expanduser('../../Data/shapes/')
patagonia = os.path.join(work_dir, 'patagonia_nueva.shp')
patagonia_grande = geemap.shp_to_ee(patagonia)
Map.addLayer(patagonia_grande, {}, 'Patagonia')
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [4]:
work_dir = os.path.expanduser('../../Data/shapes/')
norte = os.path.join(work_dir, 'area_norte.shp')
patagonia_norte = geemap.shp_to_ee(norte)
Map.addLayer(patagonia_norte, {}, 'norte')

In [5]:
work_dir = os.path.expanduser('../../Data/shapes/')
centro = os.path.join(work_dir, 'area_centro.shp')
patagonia_centro = geemap.shp_to_ee(centro)
Map.addLayer(patagonia_centro, {}, 'centro')

In [6]:
work_dir = os.path.expanduser('../../Data/shapes/')
sur = os.path.join(work_dir, 'area_sur.shp')
patagonia_sur = geemap.shp_to_ee(sur)
Map.addLayer(patagonia_grande, {}, 'sur')

## Funciones

In [7]:
def addNDVI(image): 
    ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

In [8]:
def addNDWI(image):
    ndwi = image.normalizedDifference(['B3','B5']).rename('NDWI')
    return image.addBands(ndwi)

In [9]:
def addNDGI(image):
    ndgi = image.normalizedDifference(['B3','B4']).rename('NDGI')
    return image.addBands(ndgi)

In [10]:
def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('date').toInt())

In [11]:
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

In [12]:
def Cloud50(image):
    scored = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.updateMask(scored.select(['cloud']).lt(50))  

In [13]:
def Cloud30(image):
    scored = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.updateMask(scored.select(['cloud']).lt(30)) 

In [14]:
def Agua(image):
    return image.updateMask(ndwi.lte(-0.1))

In [15]:
def Agua2021(image):
    return image.updateMask(ndwi2021.gte(0.1))

In [16]:
def Agua2020(image):
    return image.updateMask(ndwi2020.gte(0))

In [17]:
def Agua2019(image):
    return image.updateMask(ndwi2019.gte(0))

In [18]:
def Ndvi2021(image):
    return image.updateMask(ndvi2021.gt(0.089))

In [19]:
def Ndvi2020(image):
    return image.updateMask(ndvi2020.lt(0.2))

In [20]:
def Ndvi2019(image):
    return image.updateMask(ndvi2019.lt(0.2))

In [21]:
def Ndvi2(image):
    return image.updateMask(ndvi.gte(0.2))

### Funciones SUR

In [22]:
def Ndvi2021_sur(image):
    return image.updateMask(ndvi2021.gt(0.040))

In [23]:
def Agua2021_sur(image):
    return image.updateMask(ndwi2021.lt(0.1))

## Colecciones

### Verano 2021

In [24]:
start_date = '2020-10-01'
end_date = '2021-03-31'

l8_2021 = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [25]:
vizParams = {
  'bands': ['B5', 'B4', 'B3'],
  'min': 0,
  'max': 0.5,
  'gamma': [0.95, 1.1, 1]
}

In [26]:
coleccion2021 = l8_2021.map(addNDVI).map(addNDWI).map(addNDGI).map(addDate)

#### Verano Sur

In [27]:
start_date = '2019-10-01'
end_date = '2021-03-31'

l8_2021_sur = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filterBounds(patagonia_sur) \
    .filterDate(start_date, end_date)

In [28]:
vizParams = {
  'bands': ['B5', 'B4', 'B3'],
  'min': 0,
  'max': 0.5,
  'gamma': [0.95, 1.1, 1]
}

In [29]:
coleccionsur = l8_2021_sur.map(addNDVI).map(addNDWI).map(addNDGI).map(addDate)

### Verano 2020

In [30]:
start_date = '2019-10-01'
end_date = '2020-03-31'

l8_2020 = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [31]:
coleccion2020 = l8_2020.map(addNDVI).map(addNDWI).map(addNDGI).map(addDate)

### Verano 2019

In [32]:
start_date = '2018-10-01'
end_date = '2019-03-31'

l8_2019 = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [33]:
coleccion2019 = l8_2019.map(addNDVI).map(addNDWI).map(addNDGI).map(addDate)

## Quality Mosaic NDVI

In [34]:
NDVI_2021 = coleccion2021.qualityMosaic('NDVI')
NDVI_2020 = coleccion2020.qualityMosaic('NDVI')
NDVI_2019 = coleccion2019.qualityMosaic('NDVI')

In [35]:
NDVI_sur = coleccionsur.qualityMosaic('NDVI')

In [36]:
visParams = {
  'bands': ['B5', 'B4', 'B3'],
  'min': 0,
  'max': 0.4,
    'gamma': [0.95, 1.1, 1],
}

Map.addLayer(NDVI_2019, visParams, 'Greenest 2019')
Map.addLayer(NDVI_2020, visParams, 'Greenest 2020')
Map.addLayer(NDVI_2021, visParams, 'Greenest 2021')
Map.addLayer(NDVI_sur, visParams, 'Greenest sur')
Map.centerObject(patagonia_grande, 6)
Map

Map(center=[-47.891474135280234, -68.82549165230998], controls=(WidgetControl(options=['position', 'transparen…

In [37]:
ndvi2021 = NDVI_2021.select('NDVI')
ndvi2020 = NDVI_2020.select('NDVI')
ndvi2019 = NDVI_2019.select('NDVI')
ndvisur = NDVI_sur.select('NDVI')
palette = ['#d73027', '#f46d43', '#fdae61', '#fee08b', '#d9ef8b', '#a6d96a', '#66bd63', '#1a9850']

In [38]:
Map.addLayer(ndvi2021, {'palette': palette}, 'NDVI_2021')
Map.addLayer(ndvi2020, {'palette': palette}, 'NDVI_2020')
Map.addLayer(ndvi2019, {'palette': palette}, 'NDVI_2019')
Map.addLayer(ndvisur,{'palette': palette}, 'NDVI_sur')
Map

The history saving thread hit an unexpected error (OperationalError('disk I/O error')).History will not be written to the database.


Map(center=[-47.891474135280234, -68.82549165230998], controls=(WidgetControl(options=['position', 'transparen…

## Agua

### Verano 2021

In [39]:
aquast_2021 = coleccion2021.qualityMosaic('NDWI')
aquast_sur = coleccionsur.qualityMosaic('NDWI')

### Verano 2020

In [40]:
aquast_2020 = coleccion2020.qualityMosaic('NDWI')

### Verano 2019

In [41]:
aquast_2019 = coleccion2019.qualityMosaic('NDWI')

### Quality Mosaic NDWI

In [42]:
visParams = {
  'bands': ['B4', 'B3', 'B1'],
  'min': 0,
  'max': 0.4,
    'gamma': [0.95, 1.1, 1],
}

Map2.addLayer(aquast_2019, visParams, 'Agua 2019')
Map2.addLayer(aquast_2020, visParams, 'Agua 2020')
Map2.addLayer(aquast_2021, visParams, 'Agua 2021')
Map2.addLayer(aquast_sur, visParams, 'Agua sur')
Map2.centerObject(patagonia_grande, 6)
Map2

Map(center=[-47.891474135280234, -68.82549165230998], controls=(WidgetControl(options=['position', 'transparen…

In [43]:
ndwi2021 = aquast_2021.select('NDWI')
ndwi_viz = ['00FFFF', '0000FF']
ndwi2020 = aquast_2020.select('NDWI')
ndwi2019 = aquast_2019.select('NDWI')
ndwisur = aquast_sur.select('NDWI')
Map2.addLayer(ndwi2021, {'palette': ndwi_viz}, 'NDWI_2021')
Map2.addLayer(ndwi2020, {'palette': ndwi_viz}, 'NDWI_2020')
Map2.addLayer(ndwi2019, {'palette': ndwi_viz}, 'NDWI_2019')
Map2.addLayer(ndwisur, {'palette': ndwi_viz}, 'NDWI_sur')
Map2

Map(center=[-47.891474135280234, -68.82549165230998], controls=(WidgetControl(options=['position', 'transparen…

In [44]:
ndwi_rgb = ndwi2021.visualize(**{
    'min': 0.5,
    'max':1,
    'palette': ['00FFFF', '0000FF']
})

In [45]:
ndwi_mask = ndwi2021.updateMask(ndwi2021.gte(0.4))

In [46]:
map_ndwi_mask = folium.Map(location = [-45, -73], zoom_start = 5)
map_ndwi_mask.add_ee_layer(ndwi_mask, {} , 'NDWI sin tierra')
display(map_ndwi_mask)

In [47]:
ndwi_mask2020 = ndwi2020.updateMask(ndwi2020.gte(0.4))

In [48]:
map_ndwi_mask2020 = folium.Map(location = [-45, -73], zoom_start = 5)
map_ndwi_mask2020.add_ee_layer(ndwi_mask2020, {}, 'NDWI sin tierra')
display(map_ndwi_mask2020)

## Máscaras

### Verano 2021

#### Nubes 50

In [49]:
coleccion21 = coleccion2021.map(Cloud50)
greenest21 = coleccion21.qualityMosaic('NDVI')
map_2021 = folium.Map(location = [-45, -73], zoom_start = 5)
map_2021.add_ee_layer(greenest21, vizParams, 'Mapa sin Nubes')
display(map_2021)

#### Nubes 30

In [50]:
coleccion21 = coleccion2021.map(Cloud30)
greenest21 = coleccion21.qualityMosaic('NDVI')
map_2021 = folium.Map(location = [-45, -73], zoom_start = 5)
map_2021.add_ee_layer(greenest21, vizParams, 'Mapa sin Nubes')
display(map_2021)

#### Nubes Sur

In [51]:
coleccionsur = coleccionsur.map(Cloud30)
greenestsur = coleccionsur.qualityMosaic('NDVI')
map_sur = folium.Map(location = [-45, -73], zoom_start = 5)
map_sur.add_ee_layer(greenestsur, vizParams, 'Mapa sin Nubes')
display(map_sur)

In [52]:
coleccionsur2 = coleccionsur.map(Cloud50)
greenestsur2 = coleccionsur2.qualityMosaic('NDVI')
map_sur2 = folium.Map(location = [-45, -73], zoom_start = 5)
map_sur2.add_ee_layer(greenestsur2, vizParams, 'Mapa sin Nubes')
display(map_sur2)

#### Filtro de agua

In [53]:
coleccion21_ = coleccion2021.map(Cloud50)
coleccion21_ = coleccion21_.map(Agua2021)
greenest21_ = coleccion21_.qualityMosaic('NDVI')
map_2021_ = folium.Map(location = [-45, -73], zoom_start = 5)
map_2021_.add_ee_layer(greenest21_, vizParams, 'Mapa sin Nubes')
display(map_2021_)

In [54]:
agua_2021 = coleccion2021.map(Cloud50)
agua_2021 = agua_2021.map(Agua2021)
aquast21_ = agua_2021.qualityMosaic('NDWI')
map_agua2021 = folium.Map(location = [-45, -73], zoom_start = 5)
map_agua2021.add_ee_layer(aquast21_, vizParams, 'Mapa sin Nubes')
display(map_agua2021)

In [55]:
coleccion21 = coleccion2021.map(Cloud50)
greenest21 = coleccion21.qualityMosaic('NDVI')
map_2021 = folium.Map(location = [-45, -73], zoom_start = 5)
map_2021.add_ee_layer(greenest21, vizParams, 'Mapa sin Nubes')
display(map_2021)

#### Filtro NDVI

In [56]:
coleccion21 = coleccion2021.map(Cloud50)
coleccion21 = coleccion21.map(Ndvi2021)
greenest21_2 = coleccion21.qualityMosaic('NDVI')
map_2021 = folium.Map(location = [-45, -73], zoom_start = 5)
map_2021.add_ee_layer(greenest21_2, vizParams, 'Mapa sin Nubes')
display(map_2021)

In [57]:
coleccion_sur2 = coleccionsur.map(Cloud30)
coleccion_sur2 = coleccion_sur2.map(Ndvi2021_sur)
coleccion_sur2 = coleccion_sur2.map(Agua2021_sur)
greenest_sur2 = coleccion_sur2.qualityMosaic('NDVI')
map_sur2 = folium.Map(location = [-45, -73], zoom_start = 5)
map_sur2.add_ee_layer(greenest_sur2, vizParams, 'Mapa sin Nubes')
display(map_sur2)

### Verano 2020

In [58]:
coleccion20 = coleccion2020.map(Cloud50)
# coleccion2020 = coleccion2020.map(Agua)
greenest20 = coleccion20.qualityMosaic('NDVI')
map_2020 = folium.Map(location = [-45, -73], zoom_start = 5)
map_2020.add_ee_layer(greenest20, vizParams, 'Mapa sin nubes')
display(map_2020)

In [59]:
agua_2020 = coleccion2020.map(Agua2020)
aquast20_ = agua_2020.qualityMosaic('NDWI')
map_agua2020 = folium.Map(location = [-45, -73], zoom_start = 5)
map_agua2020.add_ee_layer(aquast20_, vizParams, 'Mapa sin Nubes')
display(map_agua2020)

### Verano 2019

In [60]:
coleccion19 = coleccion2019.map(Cloud50)
# coleccion2019 = coleccion2019.map(Agua)
greenest19 = coleccion19.qualityMosaic('NDVI')
map_2019 = folium.Map(location = [-45, -73], zoom_start = 5)
map_2019.add_ee_layer(greenest19, vizParams, 'Mapa sin nubes')
display(map_2019)

## Mosaicos

In [61]:
Map3.addLayer(ndvi2021, {'palette': palette}, 'NDVI_2021')
# Map3

In [62]:
mosaic = ee.ImageCollection([greenestsur,greenest21]).mosaic()

In [63]:
Map3.addLayer(mosaic, vizParams, 'Mosaico')
# Map3

In [64]:
mosaic2 = ee.ImageCollection([mosaic, aquast20_]).mosaic()

In [65]:
Map3.addLayer(mosaic2, vizParams, 'Mosaico2')
# Map3

In [66]:
mosaic3 = ee.ImageCollection([mosaic2, greenest21_2]).mosaic()

In [67]:
Map3.addLayer(mosaic3, vizParams, 'Mosaico3')
# Map3

In [68]:
mosaic4 = ee.ImageCollection([mosaic3, greenest_sur2]).mosaic()

In [69]:
Map3.addLayer(mosaic4, vizParams, 'Mosaico 4')
Map3

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [70]:
mosaic5 = ee.ImageCollection([mosaic4,aquast20_]).mosaic()

In [71]:
Map3.addLayer(mosaic5, vizParams, 'Mosaico 5')
Map3

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [73]:
mosaic_bandas = mosaic4.select(['B1','B2','B3','B4','B5','B6','B7'])
Map3.addLayer(mosaic_bandas, vizParams, 'Mosaico bandas')
Map3

Map(bottom=1318068.0, center=[-41.886943401656346, -69.0209197998047], controls=(WidgetControl(options=['posit…

## Estadísticas